In [ ]:
!git clone https://github.com/suragnair/alpha-zero-general/
!mv alpha-zero-general/* ./
!rm -rf alpha-zero-general/

Cloning into 'alpha-zero-general'...
remote: Enumerating objects: 1156, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 1156 (delta 3), reused 4 (delta 0), pack-reused 1147
Receiving objects: 100% (1156/1156), 424.24 MiB | 27.30 MiB/s, done.
Resolving deltas: 100% (635/635), done.
Updating files: 100% (123/123), done.


In [ ]:
import Arena
from MCTS import MCTS
from othello.OthelloGame import OthelloGame
from tictactoe.TicTacToeGame import TicTacToeGame
from tictactoe.TicTacToePlayers import *
from tictactoe.keras.NNet import NNetWrapper as NNet


import numpy as np
from utils import *

"""
use this script to play any two agents against each other, or play manually with
any agent.
"""

mini_othello = True  # Play in 6x6 instead of the normal 8x8.
human_vs_cpu = True

# if mini_othello:
#     g = OthelloGame(6)
# else:
#     g = OthelloGame(8)
g = TicTacToeGame()

# all players
rp = RandomPlayer(g).play
hp = HumanTicTacToePlayer(g).play



# nnet players
n1 = NNet(g)
# if mini_othello:
#     n1.load_checkpoint('./pretrained_models/othello/pytorch/','6x100x25_best.pth.tar')
# else:
#     n1.load_checkpoint('./pretrained_models/othello/pytorch/','8x8_100checkpoints_best.pth.tar')
n1.load_checkpoint('/content/pretrained_models/tictactoe/keras/', 'best-25eps-25sim-10epch.pth.tar')
args1 = dotdict({'numMCTSSims': 50, 'cpuct':1.0})
mcts1 = MCTS(g, n1, args1)
n1p = lambda x: np.argmax(mcts1.getActionProb(x, temp=0))

if human_vs_cpu:
    player2 = hp
else:

    n2 = NNet(g)
    n2.load_checkpoint('/content/pretrained_models/tictactoe/keras/', 'best-25eps-25sim-10epch.pth.tar')
    args2 = dotdict({'numMCTSSims': 50, 'cpuct': 1.0})
    mcts2 = MCTS(g, n2, args2)
    n2p = lambda x: np.argmax(mcts2.getActionProb(x, temp=0))

    player2 = n2p  # Player 2 is neural network if it's cpu vs cpu.

arena = Arena.Arena(n1p, player2, g, display=TicTacToeGame.display)

print(arena.playGames(2, verbose=True))